Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/reinforcement-learning/setup/devenv_setup.png)

# Reinforcement Learning in Azure Machine Learning - Setting Up Development Environment

Ray multi-node cluster setup requires all worker nodes to be able to communicate with the head node. This notebook explains you how to setup a virtual network, to be used by the Ray head and worker compute targets, created and used in other notebook examples.

### Prerequisite

The user should have completed the Azure Machine Learning Tutorial: [Get started creating your first ML experiment with the Python SDK](https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-1st-experiment-sdk-setup). You will need to make sure that you have a valid subscription ID, a resource group, and an Azure Machine Learning workspace.

### Azure Machine Learning SDK 
Display the Azure Machine Learning SDK version.

In [ ]:
import azureml.core

print("Azure Machine Learning SDK Version: ", azureml.core.VERSION)

### Get Azure Machine Learning workspace
Get a reference to an existing Azure Machine Learning workspace.


In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep = ' | ')

### Create Virtual Network

If you are using separate compute targets for the Ray head and worker, a virtual network must be created in the resource group.  If you have alraeady created a virtual network in the resource group, you can skip this step.

To do this, you first must install the Azure Networking API.

`pip install --upgrade azure-mgmt-network`

In [ ]:
# If you need to install the Azure Networking SDK, uncomment the following line.
#!pip install --upgrade azure-mgmt-network

In [ ]:
from azure.mgmt.network import NetworkManagementClient

# Virtual network name
vnet_name ="your_vnet"

# Default subnet
subnet_name ="default"

# The Azure subscription you are using
subscription_id=ws.subscription_id

# The resource group for the reinforcement learning cluster
resource_group=ws.resource_group

# Azure region of the resource group
location=ws.location

network_client = NetworkManagementClient(ws._auth_object, subscription_id)

async_vnet_creation = network_client.virtual_networks.create_or_update(
    resource_group,
    vnet_name,
    {
        'location': location,
        'address_space': {
            'address_prefixes': ['10.0.0.0/16']
        }
    }
)

async_vnet_creation.wait()
print("Virtual network created successfully: ", async_vnet_creation.result())

### Set up Network Security Group on Virtual Network

Depending on your Azure setup, you may need to open certain ports to make it possible for Azure to manage the compute targets that you create.  The ports that need to be opened are described [here](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-enable-virtual-network).

A common situation is that ports `29876-29877` are closed.  The following code will add a security rule to open these ports.    Or you can do this manually in the [Azure portal](https://portal.azure.com).

You may need to modify the code below to match your scenario.

In [ ]:
import azure.mgmt.network.models

security_group_name = vnet_name + '-' + "nsg"
security_rule_name = "AllowAML"

# Create a network security group
nsg_params = azure.mgmt.network.models.NetworkSecurityGroup(
    location=location,
    security_rules=[
        azure.mgmt.network.models.SecurityRule(
            name=security_rule_name,
            access=azure.mgmt.network.models.SecurityRuleAccess.allow,
            description='Reinforcement Learning in Azure Machine Learning rule',
            destination_address_prefix='*',
            destination_port_range='29876-29877',
            direction=azure.mgmt.network.models.SecurityRuleDirection.inbound,
            priority=400,
            protocol=azure.mgmt.network.models.SecurityRuleProtocol.tcp,
            source_address_prefix='BatchNodeManagement',
            source_port_range='*'
        ),
    ],
)

async_nsg_creation = network_client.network_security_groups.create_or_update(
    resource_group,
    security_group_name,
    nsg_params,
)

async_nsg_creation.wait() 
print("Network security group created successfully:", async_nsg_creation.result())

network_security_group = network_client.network_security_groups.get(
    resource_group,
    security_group_name,
)

# Define a subnet to be created with network security group
subnet = azure.mgmt.network.models.Subnet(
            id='default',
            address_prefix='10.0.0.0/24',
            network_security_group=network_security_group
            )
    
# Create subnet on virtual network
async_subnet_creation = network_client.subnets.create_or_update(
    resource_group_name=resource_group,
    virtual_network_name=vnet_name,
    subnet_name=subnet_name,
    subnet_parameters=subnet
)

async_subnet_creation.wait()
print("Subnet created successfully:", async_subnet_creation.result())

### Review the virtual network security rules
Ensure that the virtual network is configured correctly with required ports open. It is possible that you have configured rules with broader range of ports that allows ports 29876-29877 to be opened. Kindly review your network security group rules. 

In [ ]:
from files.networkutils import *

check_vnet_security_rules(ws._auth_object, ws.subscription_id, ws.resource_group, vnet_name, True)